In [9]:
import os
import torch

# Define the checkpoint directory where all models are saved
ckpt_dir = "./checkpoint_8/synthetic"

# Initialize variables to track the best model
best_acc = 0.0
best_model = None
best_model_name = None

# Loop through all files in the checkpoint directory
for filename in os.listdir(ckpt_dir):
    # print(filename)
    if filename.endswith(".pth"):  # Ensure it's a model checkpoint file
        filepath = os.path.join(ckpt_dir, filename)

        # Load the checkpoint
        checkpoint = torch.load(filepath, weights_only=False)

        # Extract the validation accuracy (acc) from the checkpoint
        acc = checkpoint.get("acc", 0.0)
        print(checkpoint["acc"])

        # Check if this model has the highest accuracy so far
        print(acc)
        if acc > best_acc:
            best_acc = acc
            best_model = checkpoint
            best_model_name = filename

# Print the best model information
print(f"The best model is: {best_model_name} with accuracy: {best_acc:.4f}")

0.8055555555555556
0.8055555555555556
0.9166666666666666
0.9166666666666666
0.9166666666666666
0.9166666666666666
0.75
0.75
0.8055555555555556
0.8055555555555556
0.75
0.75
0.8333333333333334
0.8333333333333334
0.8055555555555556
0.8055555555555556
0.19444444444444445
0.19444444444444445
0.9166666666666666
0.9166666666666666
0.8611111111111112
0.8611111111111112
0.9166666666666666
0.9166666666666666
0.75
0.75
0.9166666666666666
0.9166666666666666
0.8611111111111112
0.8611111111111112
0.8611111111111112
0.8611111111111112
0.9166666666666666
0.9166666666666666
0.8888888888888888
0.8888888888888888
0.8333333333333334
0.8333333333333334
0.9166666666666666
0.9166666666666666
0.8055555555555556
0.8055555555555556
0.9166666666666666
0.9166666666666666
0.8333333333333334
0.8333333333333334
0.9166666666666666
0.9166666666666666
0.8888888888888888
0.8888888888888888
0.7222222222222222
0.7222222222222222
0.8333333333333334
0.8333333333333334
The best model is: gin_best_clst_0.01_sep_0.0.pth with a